# **Day 3 - Conversational AI - aka Chatbot!**

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

c:\Users\Eduar\BuildAMultiModalChatBot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAJ


In [3]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [7]:
system_message = "You are a helpful assistant"

# **Please read this! A change from the video:**

In the video, I explain how we now need to write a function called:

chat(message, history)

Which expects to receive history in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```python
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in history in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function chat(message, history) where:
message is the prompt to use
history is the past conversation, in OpenAI format

We will combine the system message, history and latest message, then call OpenAI.

In [8]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

# **And then enter Gradio's magic!**

In [9]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'oi'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'oi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Oi! Como posso ajudar você hoje?', 'options': None}]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'metadata': None, 'content': 'oi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Oi! Como posso ajudar você hoje?', 'options': None}, {'role': 'user', 'content': 'Me chamo Eduardo'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'oi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Oi! Como posso ajudar você hoje?', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'Me chamo Eduardo', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Prazer em te conhecer, Eduardo! Como posso ajudar você

In [10]:

system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [12]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


In [13]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [14]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


In [15]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [16]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.
